In [5]:
import numpy as np
import cv2

green = (0,255,0)
red = (0,0,255)
white = (255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX
cap = cv2.VideoCapture(0)

#########################################################################

width = 960
height= 720
cap.set(4,height)
cap.set(3,width)

#########################################################################

def empty(a):
    pass

#########################################################################

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",1280,480)
cv2.createTrackbar("Threshold1","Parameters",70,255,empty)
cv2.createTrackbar("Threshold2","Parameters",70,255,empty)
cv2.createTrackbar("Area","Parameters",1000,10000,empty)

#########################################################################

def getContours(img,imgContour):
    
# find contours function
# retrieval method
# chanprox: all contour points
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Parameters")        
        if area > areaMin:
#             cv2.drawContours(imgContour,cnt,-1,red,7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
#             print(len(approx))
            x,y,w,h = cv2.boundingRect(approx)
    
            cv2.rectangle(imgContour,(x,y),(x+w,y+h),green,3)
            distance = -0.03*(area/100) + 37

#             # Display rectangle
#             box = cv2.boxPoints(rect)
#             box = np.int0(box)

#             cv2.circle(img,(int(x),int(y)),5,red,-1)
#             cv2.polylines(img,[box],True,(255,0,0),2)

#             cv2.putText(imgContour,"Points: " + str(len(approx)),(x+w+20,y+20),fonts,0.7,green,2)
#             cv2.putText(imgContour,"Height: " + str(int(h)),(x+w+20,y),fonts,0.7,green,2)
#             cv2.putText(imgContour,"Width: " + str(int(w)),(x+w+20,y+25),fonts,0.7,green,2)
#             cv2.putText(imgContour,"Aspect: " + str(round((h/w),1)),(x+w+20,y+25),fonts,0.7,green,2)
#             cv2.putText(imgContour,"Distance: " + str(int(distance)),(x+w+20,y+25),fonts,0.7,green,2)
            cv2.putText(imgContour,"Area: " + str(int(area)),(x+w+20,y+50),fonts,0.7,green,2)
        
            if(area < 8300):
                cv2.putText(imgContour,"Grasp: Pinch",(x+w+20,y+75),fonts,0.7,green,2)
            elif(area > 15000):
                cv2.putText(imgContour,"Grasp: Power",(x+w+20,y+75),fonts,0.7,green,2)
            else:
                cv2.putText(imgContour,"Grasp: Tripod",(x+w+20,y+75),fonts,0.7,green,2)
    
#########################################################################    

while(True):        
    success, img = cap.read()    # Capture frame-by-frame
    imgContour = img
    
# imgBlur = cv2.GaussianBlur(img,(7,7),1)
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    
# Canny edge detector is an edge detection operator that uses a multi-stage algorithm
# to detect a wide range of edges in images

# canny image takes 2 thresholds (upper and lower)
# If a pixel gradient is higher than the upper threshold, it is accepted as an edge.
# If it is between the two thresholds, it will be accepted only if it is connected
# to a pixel that is above the upper threshold.

# get trackbar position from the parameters window
    threshold1 = cv2.getTrackbarPos("Threshold1","Parameters")
    threshold2 = cv2.getTrackbarPos("Threshold2","Parameters")
    imgCanny = cv2.Canny(imgGray,threshold1,threshold2)
    
# dilation function to remove overlaps and noise    
    kernel = np.ones((5,5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
    
# contour: outline representing or bounding the shape    
    getContours(imgDil, imgContour)
    
#     imgStack = stackImages(0.8,([img, imgBlur, imgGray], [imgDil,imgDil,imgDil]))

    cv2.imshow('frame',imgContour)
    
#########################################################################    

    if cv2.waitKey(1)==27: break    # esc

cap.release()
cv2.destroyAllWindows()